## Ingestion del archivo "language.csv"

#### `Paso 0 - Actualización de variables y funciones

In [0]:
dbutils.widgets.text("p_environment", "")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

#### Paso 1 - Leer el archivo CSV usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DataType

In [0]:
languages_schema = StructType(fields=[
    StructField("LanguageID", IntegerType(), False), \
    StructField("LanguageCode", StringType(), True), \
    StructField("LanguageName", StringType(), True) \
])

In [0]:
languages_df = spark.read \
    .option("header", True) \
    .schema(languages_schema) \
    .csv(f"{bronze_folder_path}/{v_file_date}/language.csv")

#### Paso 2 - Seleccionar sólo las columnas "requeridas"

In [0]:
from pyspark.sql.functions import col

languages_selected_df = languages_df.select(
                            col("LanguageID"),
                            col("LanguageName"))

#### Paso 3 - Cambiar el nombre de las columnas según lo "requerido"

In [0]:
# Alterando somente os nomes das colunas necessárias
languages_renamed_df = languages_selected_df \
                    .withColumnRenamed("LanguageID", "language_id") \
                    .withColumnRenamed("LanguageName", "language_name")

#### Paso 4 - Agregar la columna "ingestion_date" al DataFrame

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
languages_final_df = add_ingestion_date(languages_renamed_df) \
                    .withColumn("enviroment", lit(v_environment)) \
                    .withColumn("file_date", lit(v_file_date)) 

#### Paso 5 - Escribir datos en el datalake en formato parquet

In [0]:
#languages_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/languages/")
#languages_final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_silver.languages")
languages_final_df.write.mode("overwrite").format("delta").saveAsTable("movie_silver.languages")

In [0]:
#%fs
#ls /mnt/moviehistoryadilmor/silver/languages

In [0]:
#display(spark.read.parquet(f"{silver_folder_path}/languages"))
#spark.sql("SELECT * FROM movie_silver.languages").display()

In [0]:
dbutils.notebook.exit("Success")